In [1]:
import os
import numpy as np
import pandas as pd
import napari
from glob import glob
import json
from tqdm.notebook import tqdm, trange
import sys
sys.path.append(r'../')

from tools import imgLoad
from tools import ioTools as tio
from SegTools import Blocks, Segmentation, Proofread

%gui qt

C:\Users\Yannan\Anaconda3\envs\cellCount38\lib\site-packages\napari\__init__.py:44: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.7. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)
C:\Users\Yannan\Anaconda3\envs\cellCount38\lib\site-packages\skimage\io\manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
basedir = r'Y:\People\Yannan\Segmentation\DrugTH10_30mgkg\CKet11823'
allCells = Proofread(basedir + r'\fullCells', []).cells
annotation = np.array(imgLoad.loadVirtualTifSeq(r'W:\People\Yannan\Segmentation\DF16\ABA\annotation\*.tif'))
hemisphere = np.array(imgLoad.loadVirtualTifSeq(r"W:\People\Yannan\Segmentation\DF16\ABA\hemisphere\*.tif"))

brainHierarchy = json.loads(open(r'W:\People\Malika\highResMapped_to_CAvgR1\AllenbrainAtlas\brainHierarchy.json').read())['msg'][0]#['children']#[0]
print('This is only: ', brainHierarchy['name'])

(1392, 990, 570)
(1392, 990, 570)
This is only:  root


In [3]:
# https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-dictionaries-and-lists
def regions_at_certain_level(d, level):
    if 'st_level' in d and d['st_level'] == level:
        yield {'name':d['name'],
               'id': d['id'],
               'acronym':d['acronym'], 
               'children':d['children']}
    for k in d:
        if isinstance(d[k], list):
            for i in d[k]:
                for j in regions_at_certain_level(i, level):
                    yield j

def subregions_below_certain_region(d):
    if 'name' in d:
        try:
            yield {'name':d['name'], 
                   'id': d['id'],
                   'acronym':d['acronym'],
                   'parent_structure_id':d['parent_structure_id']}
        except:
            yield {'name':d['name'], 
                   'id': d['id'],
                   'acronym':d['acronym'],
                   'parent_structure_id': 0}
    for k in d:
        if isinstance(d[k], list):
            for i in d[k]:
                for j in subregions_below_certain_region(i):
                    yield j

   

In [4]:
cellPrimaryRegionID = np.zeros(len(allCells))
cellHemisphere = np.zeros(len(allCells))
for i in trange(len(allCells)):
    x, y, z = int(allCells.iloc[i].x/4), int(allCells.iloc[i].y/4), int(allCells.iloc[i].z/2)
    cellPrimaryRegionID[i] = annotation[x, y, z]
    cellHemisphere[i] = hemisphere[x, y, z] > 0

# Exclude cells outside brain
allCells = allCells.iloc[cellPrimaryRegionID > 0]
cellHemisphere = cellHemisphere[cellPrimaryRegionID > 0] 
cellPrimaryRegionID = cellPrimaryRegionID[cellPrimaryRegionID > 0]


  0%|          | 0/28376 [00:00<?, ?it/s]

In [5]:
cells = pd.concat([allCells.reset_index().iloc[:,2:], pd.DataFrame({'primaryRegionID': cellPrimaryRegionID})], axis = 1)
cells['isLeft'] = cellHemisphere
for hierarchy in trange(1,9):
    regions = list(regions_at_certain_level(brainHierarchy, hierarchy))


    cellRegionID = np.zeros(len(cellPrimaryRegionID))
    cellRegionName = [''] * len(cellPrimaryRegionID)
    cellRegionAcronym = [''] * len(cellPrimaryRegionID)
    for i in range(len(regions)):
        subregions = list(subregions_below_certain_region(regions[i]))
        for j in range(len(subregions)):
            try:
                cellRegionID[cellPrimaryRegionID == subregions[j]['id']] = regions[i]['id']

                for selected in np.where(cellPrimaryRegionID == subregions[j]['id'])[0]:
                    cellRegionName[selected] = regions[i]['name']
                    cellRegionAcronym[selected] = regions[i]['acronym']
            except:
                1
    cells = pd.concat([cells, pd.DataFrame({'level{}RegionID'.format(hierarchy): cellRegionID,
                                           'level{}RegionName'.format(hierarchy): cellRegionName,
                                           'level{}RegionAcronym'.format(hierarchy): cellRegionAcronym})], axis = 1)  

cells = cells.loc[cells.level1RegionID > 0].reset_index().iloc[:,1:]


  0%|          | 0/8 [00:00<?, ?it/s]

In [6]:
tio.mkdirs(basedir + r'\fullCells\finalResults')
cells.to_csv(basedir + r'\fullCells\finalResults\finalBlobsWithRegions.csv')

Create  Y:\People\Yannan\Segmentation\DrugTH10_30mgkg\CKet11823\fullCells\finalResults
